<a href="https://www.kaggle.com/code/avtnshm/clinical-modernbert-v-biomedicalmb-on-ddxplus-data?scriptVersionId=254542935" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

# Load the train, test, and validate datasets
train_df = pd.read_csv('/kaggle/input/mldataset/ddxplus/train.csv')
test_df = pd.read_csv('/kaggle/input/mldataset/ddxplus/test.csv')
validate_df = pd.read_csv('/kaggle/input/mldataset/ddxplus/validate.csv')

# Display the first few rows of the train dataset
train_df.head()

In [ ]:

import json

# Load JSON files
with open('/kaggle/input/mldataset/ddxplus/release_conditions.json', 'r') as f:
    conditions = json.load(f)
with open('/kaggle/input/mldataset/ddxplus/release_evidences.json', 'r') as f:
    evidences = json.load(f)

# Convert JSON to DataFrames for easier inspection (optional)
conditions_df = pd.DataFrame.from_dict(conditions, orient='index')
evidences_df = pd.DataFrame.from_dict(evidences, orient='index')

# Display the first few rows
print("Conditions (Diseases):")
print(conditions_df.head())
print("\nEvidences (Symptoms):")
print(evidences_df.head())

In [ ]:
def map_symptom_codes(symptom_dict, evidences_df):
    return [evidences_df.loc[code]["question_en"] for code in symptom_dict.keys() if code in evidences_df.index]

conditions_df["symptom_questions"] = conditions_df["symptoms"].apply(lambda x: map_symptom_codes(x, evidences_df))
conditions_df["antecedent_questions"] = conditions_df["antecedents"].apply(lambda x: map_symptom_codes(x, evidences_df))

In [ ]:
print(conditions_df[["condition_name", "symptom_questions", "antecedent_questions"]].head())

In [ ]:
# Map PATHOLOGY to condition_name
train_df['disease_name'] = train_df['PATHOLOGY'].map(lambda x: conditions.get(x, {}).get('condition_name', x))

# Display the result
print(train_df[['PATHOLOGY', 'disease_name']].head())

In [ ]:
# Delete the symptom_questions and antecedent_questions columns
conditions_df = conditions_df.drop(columns=["symptom_questions", "antecedent_questions"], errors="ignore")

# Verify the columns are removed
print(conditions_df.columns)

In [ ]:
def map_symptom_codes(symptom_dict, evidences_df):
    # Return list of English questions for valid symptom codes
    return [evidences_df.loc[code, "question_en"] for code in symptom_dict.keys() if code in evidences_df.index]

# Apply to symptoms and antecedents in conditions_df
conditions_df["symptom_questions"] = conditions_df["symptoms"].apply(lambda x: map_symptom_codes(x, evidences_df))
conditions_df["antecedent_questions"] = conditions_df["antecedents"].apply(lambda x: map_symptom_codes(x, evidences_df))

# Display the result
print(conditions_df[["condition_name", "symptom_questions", "antecedent_questions"]].head())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot top 10 diseases
plt.figure(figsize=(10, 6))
train_df['disease_name'].value_counts().head(10).plot(kind='bar', color='#1f77b4')
plt.title('Top 10 Most Frequent Diseases')
plt.xlabel('Disease')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Column names from conditions_df
print("Conditions DataFrame columns:")
print(conditions_df.columns.tolist())

# Column names from evidences_df
print("\nEvidences DataFrame columns:")
print(evidences_df.columns.tolist())


print("Train columns:", train_df.columns.tolist())
print("Validate columns:", validate_df.columns.tolist())
print("Test columns:", test_df.columns.tolist())

In [ ]:
!pip install transformers -q
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
CMB = AutoModel.from_pretrained('Simonlee711/Clinical_ModernBERT')
tokenizer1 = AutoTokenizer.from_pretrained('Simonlee711/Clinical_ModernBERT')
CMB.__class__

In [ ]:
from transformers import AutoModel

BioClinical_ModernBERT = "thomas-sounack/BioClinical-ModernBERT-base"

tokenizer2 = AutoTokenizer.from_pretrained(BioClinical_ModernBERT)
BMB = AutoModel.from_pretrained(BioClinical_ModernBERT)

BMB.__class__

In [ ]:
print(train_df['DIFFERENTIAL_DIAGNOSIS'].nunique())
print(test_df['DIFFERENTIAL_DIAGNOSIS'].nunique())
validate_df['DIFFERENTIAL_DIAGNOSIS'].nunique()

In [49]:
import torch

# Helper: get English symptom questions from evidence codes
def get_symptom_text(evidence_codes, evidences_df):
    return " ".join([
        evidences_df.loc[code]["question_en"]
        for code in evidence_codes.split(';') if code in evidences_df.index
    ])

# 1. Create text column for embedding input
train_df['symptom_text'] = train_df['EVIDENCES'].apply(lambda x: get_symptom_text(x, evidences_df))

# 2. Function to get CLS embedding from a model
def get_cls_embedding(text_list, tokenizer, model):
    model.eval()
    embeddings = []

    for text in text_list:
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token
        embeddings.append(cls_embedding.squeeze().numpy())
    
    return np.array(embeddings)

# 3. Generate embeddings for both models
print("Generating embeddings from Clinical_ModernBERT (CMB)...")
cmb_embeddings = get_cls_embedding(train_df['symptom_text'], tokenizer1, CMB)

print("Generating embeddings from BioClinical_ModernBERT (BMB)...")
bmb_embeddings = get_cls_embedding(train_df['symptom_text'], tokenizer2, BMB)

# 4. Save embeddings if needed (optional)
np.save("/kaggle/working/cmb_embeddings.npy", cmb_embeddings)
np.save("/kaggle/working/bmb_embeddings.npy", bmb_embeddings)

print("Embedding shapes — CMB:", cmb_embeddings.shape, "| BMB:", bmb_embeddings.shape)


Generating embeddings from Clinical_ModernBERT (CMB)...


KeyboardInterrupt: 

In [52]:
!nvidia-smi


Wed Aug  6 09:15:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [51]:
import torch
import numpy as np

# Helper: get English symptom questions from evidence codes
def get_symptom_text(evidence_codes, evidences_df):
    return " ".join([
        evidences_df.loc[code]["question_en"]
        for code in evidence_codes.split(';') if code in evidences_df.index
    ])

# Take a small sample of 100 rows
sample_df = train_df.sample(n=100, random_state=42).copy()

# Create text column for embedding input
sample_df['symptom_text'] = sample_df['EVIDENCES'].apply(lambda x: get_symptom_text(x, evidences_df))

# Function to get CLS embedding from a model
def get_cls_embedding(text_list, tokenizer, model):
    model.eval()
    embeddings = []

    for text in text_list:
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token
        embeddings.append(cls_embedding.squeeze().numpy())

    return np.array(embeddings)

# Generate embeddings for both models
print("Generating embeddings from Clinical_ModernBERT (CMB)...")
cmb_embeddings = get_cls_embedding(sample_df['symptom_text'], tokenizer1, CMB)

print("Generating embeddings from BioClinical_ModernBERT (BMB)...")
bmb_embeddings = get_cls_embedding(sample_df['symptom_text'], tokenizer2, BMB)

# Save embeddings (optional)
np.save("/kaggle/working/cmb_embeddings_sample.npy", cmb_embeddings)
np.save("/kaggle/working/bmb_embeddings_sample.npy", bmb_embeddings)

print("Embedding shapes — CMB:", cmb_embeddings.shape, "| BMB:", bmb_embeddings.shape)


Generating embeddings from Clinical_ModernBERT (CMB)...
Generating embeddings from BioClinical_ModernBERT (BMB)...
Embedding shapes — CMB: (100, 768) | BMB: (100, 768)
